In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


SHUFFLE_BUFFER = 500
BATCH_SIZE = 10

Test giusto per scrivere qualcosa! 

In [2]:
'''
print(tf.version)
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
'''

print('let\'s try this out!')
df = pd.read_csv('./file/txCollTrendTimeseries.csv')

let's try this out!


In [3]:
#df = df.drop('Attributes', axis=1)
df = df.drop('Unnamed: 0', axis=1)


In [4]:
#df.head()
#df.dtypes
target = df.pop('Price')



In [5]:
numeric_feature_names = ['CollectionTrend', 'Price-1',  'Price-2', 'Price-3','Price-4','Price-5','Price-6','Price-7','Price-8','Price-9','Price-10']
numeric_features = df[numeric_feature_names]
numeric_features.head()

,CollectionTrend,Price-1,Price-2,Price-3,Price-4,Price-5,Price-6,Price-7,Price-8,Price-9,Price-10
0,-1,0,0,0,0,0,0,0,0,0,0
1,-1,1000000000,0,0,0,0,0,0,0,0,0
2,-1,1000000000,1000000000,0,0,0,0,0,0,0,0
3,-1,1230000000,1000000000,1000000000,0,0,0,0,0,0,0
4,-1,1200000000,1230000000,1000000000,1000000000,0,0,0,0,0,0


In [6]:
tf.convert_to_tensor(numeric_features)

2022-08-03 11:33:19.299438: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-08-03 11:33:19.333743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-08-03 11:33:19.334210: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-08-03 11:33:19.335880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

<tf.Tensor: shape=(5206152, 11), dtype=int64, numpy=
array([[        -1,          0,          0, ...,          0,          0,
                 0],
       [        -1, 1000000000,          0, ...,          0,          0,
                 0],
       [        -1, 1000000000, 1000000000, ...,          0,          0,
                 0],
       ...,
       [         0, 2300000000, 2500000000, ..., 3000000000, 3000000000,
        3990000000],
       [         0, 2300000000, 2300000000, ..., 3000000000, 3000000000,
        3000000000],
       [         0, 2300000000, 2300000000, ..., 2990000000, 3000000000,
        3000000000]])>

In [7]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)

2022-08-03 11:33:20.963364: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 458141376 exceeds 10% of free system memory.
2022-08-03 11:33:21.203597: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 458141376 exceeds 10% of free system memory.


In [8]:
normalizer(numeric_features.iloc[:3])

<tf.Tensor: shape=(3, 11), dtype=float32, numpy=
array([[-0.7173765 , -0.2438479 , -0.24384217, -0.24382915, -0.2438179 ,
        -0.24380066, -0.24378408, -0.24376582, -0.24374713, -0.24373311,
        -0.24371679],
       [-0.7173765 , -0.15271129, -0.24384217, -0.24382915, -0.2438179 ,
        -0.24380066, -0.24378408, -0.24376582, -0.24374713, -0.24373311,
        -0.24371679],
       [-0.7173765 , -0.15271129, -0.15269123, -0.24382915, -0.2438179 ,
        -0.24380066, -0.24378408, -0.24376582, -0.24374713, -0.24373311,
        -0.24371679]], dtype=float32)>

In [9]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [10]:
numeric_dataset = tf.data.Dataset.from_tensor_slices((numeric_features, target))

for row in numeric_dataset.take(3):
  print(row)

2022-08-03 11:37:27.859536: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 458141376 exceeds 10% of free system memory.


(<tf.Tensor: shape=(11,), dtype=int64, numpy=array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])>, <tf.Tensor: shape=(), dtype=int64, numpy=1000000000>)
(<tf.Tensor: shape=(11,), dtype=int64, numpy=
array([        -1, 1000000000,          0,          0,          0,
                0,          0,          0,          0,          0,
                0])>, <tf.Tensor: shape=(), dtype=int64, numpy=1000000000>)
(<tf.Tensor: shape=(11,), dtype=int64, numpy=
array([        -1, 1000000000, 1000000000,          0,          0,
                0,          0,          0,          0,          0,
                0])>, <tf.Tensor: shape=(), dtype=int64, numpy=1230000000>)


In [11]:
numeric_batches = numeric_dataset.shuffle(6000000).batch(BATCH_SIZE)


model = get_basic_model()
model.fit(numeric_features, target, epochs=2)
#model.fit(numeric_features, target, epochs=15, batch_size=BATCH_SIZE)


Epoch 1/2
    39/162693 [..............................] - ETA: 11:07 - loss: -35948376064.0000 - accuracy: 0.0000e+00

2022-08-03 11:37:30.759340: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


162693/162693 [==============================] - 708s 4ms/step - loss: -15176646959122350080.0000 - accuracy: 1.2120e-04
Epoch 2/2
162693/162693 [==============================] - 717s 4ms/step - loss: -203701153661315973120.0000 - accuracy: 1.2120e-04
